In [33]:

from IPython.core.display import HTML
import jieba


class SearcherScan():
    def __init__(self, title_file):
        with open(title_file, 'r') as f:
            titles = f.read()
        self.title_list = list(set(titles.split()))

    def word_match(self, words, title):
        ifmatch = True
        keyword_ = ' '.join(jieba.cut(words))
        for word in keyword_.split():
            if word != ' ' and word not in title:
                ifmatch = False
        return ifmatch

    def conv_query(self, query):
        query_new_parts = []
        for part in list(jieba.cut(query)):
            if part == '(' or part == ')':
                query_new_parts.append(part)
            elif part in ('and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                query_new_parts.append(part.lower())
            else:
                query_new_parts.append("self.word_match('{}',title)".format(part))
        query_new = ''.join(query_new_parts)
        return query_new

    def highlighter(self, doc, word):
        for part in list(jieba.cut(word)):
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    
    def search(self, query):
        query_new = self.conv_query(query)
        for title in self.title_list:
            if eval(query_new):
                display(HTML(self.highlighter(title, query)))


query = '苹果 and (芯片 or 高通)'
#query_new_parts=[]
#for part in list(jieba.cut(query)):
#    if part=='(' or part ==')':
#        query_new_parts.append(part)
#    elif part in ('and','AND','or','OR','NOT','not',' '):
#            query_new_parts.append(part.lower())
#    else:
#        query_new_parts.append("word_match('{},title')".format(part))
#query_new=''.join(query_new_parts)
searcher = SearcherScan('titles.txt')
searcher.search(query)

from IPython.core.display import HTML
import jieba

class SearcherIIndex():
     def __init__(self, docs_file): 
            self.index = dict()    
            self.max_id=0
            self.doc_list = [] 
        
             with open(docs_file, 'r') as f:
            docs_data = f.read()
        
                for doc in docs_data.split():
            self.add_doc(doc)

    def add_doc(self, doc):
        self.doc_list.append(doc)
        for term in list(jieba.cut(doc)):
            #构建和更新各Term对应的Posting(集合)
            if term in self.index: 
                self.index[term].add(self.max_id)
            else:
                self.index[term] = set([self.max_id])
        self.max_id += 1
        return self.max_id - 1
    
    def word_match(self, word):
        result = None
        for term in list(jieba.cut(word)):
            if result is None:
                result = self.index.get(term, set())
            else:
                result = result & self.index.get(term, set())
        if result is None:
            result = set()
        return result

    def conv_query(self, query):
        query_new_parts = []
        all_parts = list(jieba.cut(query))
        idx = 0
        cache = '' #缓存变量，用于回收分词过程被切开的短语片段
        count_parts = len(all_parts)
        while idx < count_parts:
            if all_parts[idx] == '(' or all_parts[idx] == ')':
                query_new_parts.append(all_parts[idx])
            elif all_parts[idx] == ' ':
                query_new_parts.append(' ')
            elif all_parts[idx] in ('and', 'AND'):
                query_new_parts.append('&')
            elif all_parts[idx] in ('or', 'OR'):
                query_new_parts.append('|')
            elif all_parts[idx] in ('not', 'NOT',):
                query_new_parts.append('-')
            elif idx + 1 < count_parts and all_parts[idx+1] not in (' ', ')'): #被分词切开的短语部分回收至缓存
                cache += all_parts[idx]
            else:
                query_new_parts.append("self.word_match('{}')".format(cache + all_parts[idx]))
                cache = '' #合并完成清空缓存
            idx += 1
        query_new = ''.join(query_new_parts)
        for query_new_parts in query_news:
            if query_new_parts ==' ':
                query_new_parts.replace(' ','')             
        return query_new
    
    

    def highlighter(self, doc, word):
        for part in list(jieba.cut(word)):
            #TODO(CHG):短语高亮需要先分词
            if part not in ('(', ')', 'and', 'AND', 'or', 'OR', 'NOT', 'not', ' '):
                doc = doc.replace(part, '<span style="color:red">{}</span>'.format(part))
        return doc

    def search(self, query):
        result = []
        query_new = self.conv_query(query)
        for did in eval(query_new):
            result.append(self.highlighter(self.doc_list[did], query))
        return result
    
    searcher = SearcherIIndex('titles.txt')
    query = '苹果 芯片 and (三星 or 华为)'
for doc in searcher.search(query):
    display(HTML(doc))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 73)